In [ ]:
from pyglotaran_extras import plot_data_overview

experiment_data = {
    "GFPppH2O": "data/gfph2oa.ascii",
}

plot_data_overview(experiment_data["GFPppH2O"], linlog=True, linthresh=1,cmap="seismic",datamin=-50,datamax=50);


In [ ]:
from glotaran.io import load_scheme
# from glotaran.project.scheme import Scheme
from glotaran.io import load_parameters


In [ ]:
parameters = load_parameters("models/parameters_sequential2.yml", format_name="yml")


In [ ]:
parameters


In [ ]:
scheme = load_scheme("models/model_sequential_rewrite2.yml", format_name="yml")


In [ ]:
from glotaran.io import load_dataset
scheme.load_data({"GFPppH2O": load_dataset("data/gfph2oa.ascii"),"GFPpdpH2O": load_dataset("data/gfph2ob.ascii")})


In [ ]:
result = scheme.optimize(parameters,  maximum_number_function_evaluations=11)


In [ ]:
result.data["renamed_pp"]=result.data["GFPppH2O"].rename({"species_associated_estimation":"species_associated_spectra"})
result.data["renamed_pdp"]=result.data["GFPpdpH2O"].rename({"species_associated_estimation":"species_associated_spectra"})


In [ ]:
result.data["GFPppH2O"].a_matrix


In [ ]:
result.data["renamed_pp"]


In [ ]:
result.parameters_optimized


In [ ]:
# result.data["GFPppH2O"].lifetime_decay
# AttributeError: 'Dataset' object has no attribute 'lifetime_decay'


In [ ]:
result.data["GFPppH2O"].data.plot(x="time",y="spectral",size=2)


In [ ]:
# fig, axes = result.data["GFPppH2O"].data.plot(x="time",y="spectral",size=2)
result.data["GFPppH2O"].data.plot(x="time",y="spectral",size=2)
# axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction",fontsize=16)


In [ ]:
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas, plot_norm_sas
import matplotlib.pyplot as plt


def plot_concentration_and_spectra(result_dataset):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    plot_concentrations(result_dataset,axes[0], center_λ=0, linlog=True)
    plot_sas(result_dataset,axes[1])
    plot_norm_sas(result_dataset,axes[2])
    return fig, axes


In [ ]:
fig, axes = plot_concentration_and_spectra(result.data["renamed_pp"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction",fontsize=16)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction",fontsize=16)


In [ ]:
data_array = result.data["renamed_pdp"].coherent_artifact_associated_estimation

import matplotlib.pyplot as plt

# Assuming `data_array` is your DataArray
plt.plot(data_array.coords['spectral'], data_array.values[0][0])
plt.plot(data_array.coords['spectral'], data_array.values[0][1]) # is all nan
plt.xlabel('Spectral')
plt.ylabel('Value')
plt.title('2D Line Plot')
plt.show()

data_array.values[0][1][0:10]


In [ ]:
fig, axes = plot_concentration_and_spectra(result.data["renamed_pdp"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)


In [ ]:
from pyglotaran_extras.plotting.plot_traces import (
    plot_fitted_traces,
    select_plot_wavelengths,
)
from pyglotaran_extras.plotting.style import PlotStyle

wavelengths = select_plot_wavelengths(
    result.data["GFPppH2O"], equidistant_wavelengths=False, axes_shape=(4, 3)
)
fig3tr, axes = plot_fitted_traces(
    result,
    wavelengths,
    axes_shape=(4, 3),
    linlog=True,
    linthresh=1,
    cycler=PlotStyle().data_cycler_solid_dashed,
)


In [ ]:
from rich import print
print(names_to_json(extract_names_from_xarray(result.data["GFPpdpH2O"])))


In [ ]:
result.data["GFPpdpH2O"].scale


Object inspection (debugging)

In [ ]:
import xarray as xr
import json


def extract_names_from_xarray(ds: xr.Dataset) -> dict:
    return {
        "coordinate_names": list(ds.coords.keys()),
        "data_variable_names": list(ds.data_vars.keys()),
        "index_names": list(ds.indexes.keys()),
        "attribute_names": list(ds.attrs.keys()) if ds.attrs else [],
    }


def names_to_json(names: dict) -> str:
    return json.dumps(names)


def compare_xarray_names(json1: str, json2: str) -> dict:
    dict1 = json.loads(json1)
    dict2 = json.loads(json2)

    result = {}
    for key in [
        "coordinate_names",
        "data_variable_names",
        "index_names",
        "attribute_names",
    ]:
        set1 = set(dict1[key])
        set2 = set(dict2[key])

        result[key] = {
            "common": list(set1 & set2),
            "unique_to_1": list(set1 - set2),
            "unique_to_2": list(set2 - set1),
        }

    return result


In [ ]:
from rich import print
print(extract_names_from_xarray(result.data["GFPpdpH2O"]))
